In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd

import wfdb
import wfdb.processing as wp
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import signal
import shutil

import time
from scipy import sparse
import os
import warnings
warnings.filterwarnings("ignore")
import scipy.io as sio
import pywt

In [4]:
path = "/media/jdcloud/"
labels = pd.read_csv(path+"reference.csv")
labels.head()

,File_name,label1,label2,label3,label4,label5,label6,label7,label8
0,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN0003,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN0004,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAIN0005,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def read_data_labels_info(data_path, split = "Train",preprocess=True):
    """ Read data """

    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels and one-hot encode
    labels = pd.read_csv(data_path+"reference.csv")
    #2156+224+672+654+180+826+534+504+1953
    
    # Read time-series data
    channel_files = os.listdir(path_signals)
    channel_files.sort()
    n_channels = 12#len(channel_files)

    # Initiate array
    list_of_channels = []
    i_ch = 0
    
    data_x = []
    data_y = []
    
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1','II', 'aVR', 'V3', 'III', 'V5']
    channel_mid_name = ['II','aVR','V2','V5']
    channel_post_name = ['III','aVF','V3','V6']
      
    info = []
    
    for i_ch,fil_ch in enumerate(channel_files[:]):#tqdm

        labels_list = list(labels.iloc[i_ch].values[:])#.dropna()
        #print(labels_list)

        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))
        a = 0
        for m in labels_list:
            if m in [0,1,2,3,4,5,6,7,8]:
                a += 1
        labels_list.append(a)
        labels_list.append(ecg['sex'][0])
        labels_list.append(ecg['age'][0][0])
        labels_list.append(ecg['I'].shape[1])
        
        
        info.append(labels_list)
    column_name = ['File_name','label1','label2','label3','label4','label5','label6',
                   'label7','label8','labels_num','sex','age','dataSize']
    info_pd =pd.DataFrame(columns=column_name,index=range(len(channel_files)),data=info)
    # Return 
    return info_pd

In [9]:
data_info = read_data_labels_info(path)

In [10]:
data_info.head() 

,File_name,label1,label2,label3,label4,label5,label6,label7,label8,labels_num,sex,age,dataSize
0,TRAIN0001,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE,63,5000
1,TRAIN0002,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE,51,5000
2,TRAIN0003,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE,85,5000
3,TRAIN0004,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE,69,5000
4,TRAIN0005,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE,76,5000


In [11]:
#8 - 7 - 6 - 5 - 4 - 3 - 2 - 1 - 0
2156+224+672+654+180+826+534+504+1953

7703

In [12]:
data_info[(data_info.label1==7) & (data_info.labels_num==1)]['label7'].index.shape

(195,)

In [75]:
data_info[data_info.label1==7]['label7'].index.shape

(205,)

In [73]:
data_info[data_info.label2==7]['label7'].index.shape

(19,)

In [69]:
data_info[data_info.label1==4]['label4'].index.shape

(111,)

In [70]:
data_info[data_info.label2==4]['label4'].index.shape

(66,)

In [71]:
data_info[data_info.label3==4]['label4'].index.shape

(3,)

In [8]:
data_info[data_info.labels_num>1]['label1'].index.shape

(1035,)

In [63]:
data_info[data_info.label1==0]['label1'].index.shape

(1953,)

In [64]:
data_info[data_info.label1==8]['label8'].index.shape

(1547,)

In [65]:
data_info[data_info.label2==8]['label8'].index.shape

(532,)

In [66]:
data_info[data_info.label3==8]['label8'].index.shape

(71,)

In [13]:
data_info['labels_num'].value_counts()

1    5465
2     881
3     143
4       9
6       1
5       1
Name: labels_num, dtype: int64

In [14]:
5465+2*881+3*143+4*9+11

7703

In [24]:
data_info[(data_info.label1==0) & (data_info.dataSize>5000)]['dataSize'].value_counts()

7500     444
10000     26
8000       9
9000       5
8500       5
9500       5
10500      3
15000      3
7000       3
6000       2
13500      2
12500      2
11500      2
6500       2
13000      2
12000      1
11000      1
16500      1
Name: dataSize, dtype: int64

In [13]:
label2_list = data_info[data_info.labels_num==2].index.tolist()

In [14]:
pd2 = data_info.loc[label2_list]
pd2.columns = ['File_name', 'label2', 'label1', 'label3', 'label4', 'label5', 'label6',
       'label7', 'label8','labels_num','sex','age','dataSize']

In [15]:
label3_list = data_info[data_info.labels_num==3].index.tolist()
pd3_1 = data_info.loc[label3_list]
pd3_1.columns = ['File_name', 'label2', 'label1', 'label3', 'label4', 'label5', 'label6',
       'label7', 'label8','labels_num','sex','age','dataSize']

pd3_2 = data_info.loc[label3_list]
pd3_2.columns = ['File_name', 'label3', 'label2', 'label1', 'label4', 'label5', 'label6',
       'label7', 'label8','labels_num','sex','age','dataSize']

In [16]:
label4_list = data_info[data_info.labels_num==4].index.tolist()
pd4_1 = data_info.loc[label4_list]
pd4_1.columns = ['File_name', 'label2', 'label1', 'label3', 'label4', 'label5', 'label6',
       'label7', 'label8','labels_num','sex','age','dataSize']

pd4_2 = data_info.loc[label4_list]
pd4_2.columns = ['File_name', 'label3', 'label2', 'label1', 'label4', 'label5', 'label6',
       'label7', 'label8','labels_num','sex','age','dataSize']

pd4_3 = data_info.loc[label4_list]
pd4_3.columns = ['File_name', 'label4', 'label2', 'label3', 'label1', 'label5', 'label6',
       'label7', 'label8','labels_num','sex','age','dataSize']

In [17]:
labels_en = pd.concat([data_info,pd2,pd3_1,pd3_2,pd4_1,pd4_2,pd4_3])#[:6510]
labels_en.sort_index(inplace=True)#.head(8)

In [18]:
labels_en

,File_name,age,dataSize,label1,label2,label3,label4,label5,label6,label7,label8,labels_num,sex
0,TRAIN0001,63,5000,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE
1,TRAIN0002,51,5000,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE
2,TRAIN0003,85,5000,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE
3,TRAIN0004,69,5000,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE
4,TRAIN0005,76,5000,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE
5,TRAIN0006,67,5000,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE
6,TRAIN0007,78,5000,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE
7,TRAIN0008,69,5000,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE
8,TRAIN0009,91,5000,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE
9,TRAIN0010,30,5000,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE


In [62]:
labels_en[labels_en.labels_num == 3]["sex"].value_counts()

MALE      306
FEMALE    123
Name: sex, dtype: int64